In [1]:
%pylab inline
import pandas as pd

Populating the interactive namespace from numpy and matplotlib


In [2]:
df = pd.read_csv("all_processed.csv")
df.head(2)

,Unnamed: 0,Unnamed: 0.1,company,location,dates,job-title,summary,pros,cons,advice-to-mgmt,overall-ratings,work-balance-stars,culture-values-stars,carrer-opportunities-stars,comp-benefit-stars,senior-mangemnet-stars,helpful-count,link
0,0,1,google,none,"Dec 11, 2018",Current Employee - Anonymous Employee,best compani work,peopl smart friendli,bureaucraci slow thing,none,5.0,4.0,5.0,5.0,4.0,5.0,0,https://www.glassdoor.com/Reviews/Google-Revie...
1,1,2,google,"Mountain View, CA","Jun 21, 2013",Former Employee - Program Manager,move speed light burn inevit,food food food cafe main campu mtv alon mini k...,work life balanc balanc perk benefit illus kee...,1) Don't dismiss emotional intelligence and ad...,4.0,2.0,3.0,3.0,5.0,3.0,2094,https://www.glassdoor.com/Reviews/Google-Revie...


In [3]:
df = df[df["summary"].notna()]
df = df[df["pros"].notna()]
df = df[df["cons"].notna()]

In [4]:
df["text"] = df["summary"].str.cat(df["pros"].str.cat(df["cons"],sep=" "),sep=" ")

In [5]:
df[["summary","pros","cons","text"]].head()

,summary,pros,cons,text
0,best compani work,peopl smart friendli,bureaucraci slow thing,best compani work peopl smart friendli bureauc...
1,move speed light burn inevit,food food food cafe main campu mtv alon mini k...,work life balanc balanc perk benefit illus kee...,move speed light burn inevit food food food ca...
2,great balanc big compani secur fun fast move p...,softwar engin among king hill googl engin driv...,becom larger come grow pain bureaucraci slow r...,great balanc big compani secur fun fast move p...
3,best place work also demand,find well regard compani actual deserv hype ge...,live sf commut take hour hour way shuttl somet...,best place work also demand find well regard c...
4,uniqu one kind dream job,googl world everi compani lot peopl seriou gap...,work mtv hq given work howev passion want adva...,uniqu one kind dream job googl world everi com...


In [6]:
def is_happy(rating):
    if rating >= 4:
        return 1
    else:
        return 0

df["label"] = df.apply(lambda row: is_happy(row["overall-ratings"]), axis=1)

In [7]:
y = df["label"]

In [20]:
np.mean(df["summary"].str.len())

18.53376143508448

In [19]:
np.median(df["summary"].str.len())

16.0

In [10]:
from keras.preprocessing.text import Tokenizer
max_word_count = 10000
tokenizer = Tokenizer(num_words=max_word_count)
tokenizer.fit_on_texts(df["summary"] + df["pros"] + df["cons"])

Using TensorFlow backend.


In [11]:
tokenizer.word_index

{'work': 1,
 'great': 2,
 'compani': 3,
 'peopl': 4,
 'manag': 5,
 'good': 6,
 'get': 7,
 'time': 8,
 'benefit': 9,
 'lot': 10,
 'place': 11,
 'team': 12,
 'employe': 13,
 'life': 14,
 'job': 15,
 'pay': 16,
 'opportun': 17,
 'hour': 18,
 'make': 19,
 'environ': 20,
 'like': 21,
 'cultur': 22,
 'balanc': 23,
 'learn': 24,
 'product': 25,
 'amazon': 26,
 'day': 27,
 'mani': 28,
 'realli': 29,
 'custom': 30,
 'one': 31,
 'hard': 32,
 'best': 33,
 'year': 34,
 'new': 35,
 'thing': 36,
 'much': 37,
 'chang': 38,
 'career': 39,
 'smart': 40,
 'appl': 41,
 'need': 42,
 'move': 43,
 'experi': 44,
 'take': 45,
 'long': 46,
 'high': 47,
 'go': 48,
 'microsoft': 49,
 'develop': 50,
 'even': 51,
 'growth': 52,
 'also': 53,
 'project': 54,
 'promot': 55,
 'want': 56,
 'alway': 57,
 'busi': 58,
 'well': 59,
 'level': 60,
 'polit': 61,
 'challeng': 62,
 'process': 63,
 'everi': 64,
 'expect': 65,
 'amaz': 66,
 'salari': 67,
 'posit': 68,
 'feel': 69,
 'differ': 70,
 'retail': 71,
 'hire': 72,
 'trai

In [12]:
X_tokens = tokenizer.texts_to_sequences(df["summary"])

In [13]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    X_tokens, df["label"], test_size=.2, random_state=22)

In [14]:
from keras import preprocessing
maxlen = 20

X_train = preprocessing.sequence.pad_sequences(X_train, maxlen=maxlen)
X_test = preprocessing.sequence.pad_sequences(X_test, maxlen=maxlen)
X_train[:2]

array([[  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0, 161,   6],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   2,  11,   1]], dtype=int32)

In [25]:
from keras.models import Sequential
from keras.layers import Flatten, Dense, Embedding

model = Sequential()
# We specify the maximum input length to our Embedding layer
# so we can later flatten the embedded inputs
model.add(Embedding(max_word_count, 64, input_length=maxlen))
# After the Embedding layer, 
# our activations have shape `(samples, maxlen, 8)`.

# We flatten the 3D tensor of embeddings 
# into a 2D tensor of shape `(samples, maxlen * 8)`
#model.add(Flatten())

# We add the classifier on top
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='rmsprop', loss="mse")
#model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc'])
model.summary()



_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_5 (Embedding)      (None, 20, 64)            640000    
_________________________________________________________________
flatten_4 (Flatten)          (None, 1280)              0         
Total params: 640,000
Trainable params: 640,000
Non-trainable params: 0
_________________________________________________________________


In [26]:
history = model.fit(X_train, y_train,
                    epochs=10,
                    batch_size=32,
                    validation_split=0.2)

ValueError: Error when checking target: expected flatten_4 to have shape (1280,) but got array with shape (1,)

In [ ]:
model.predict(X_train[:3])

In [ ]:
maxlen = 100

In [ ]:
word_index = 0
this_dict ={}

for i, row in df.iterrows():
    row["summary"]
    